In [1]:
import tensorflow as tf
from tensorflow import keras

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam


In [2]:
dataset_path = 'D:/10th Semester/CVPR/FaceRecognition/Imagedata/train'

In [3]:
# Load dataset with resizing images to the same dimensions
def load_dataset(dataset_path):
    images = []
    labels = []
    label_names = os.listdir(dataset_path)
    for index, name in enumerate(tqdm(label_names)):
        folder_path = os.path.join(dataset_path, name)
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            image = cv2.imread(image_path)
            if image is not None:  # Check if the image was successfully loaded
                image = cv2.resize(image, (200, 200))  # Resize images to 224x224 pixels
                images.append(image)
                labels.append(index)
            else:
                print(f"Warning: Could not load image {image_path}")
    images = np.array(images, dtype='float32') / 255.0  # Normalize images
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=len(label_names))
    return images, labels, label_names

images, labels, label_names = load_dataset(dataset_path)


100%|██████████| 2/2 [00:00<00:00, 18.77it/s]


In [4]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [5]:


# def build_model(num_classes):
#     model = models.Sequential([
#         layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
#         layers.MaxPooling2D((2, 2)),
#         layers.Conv2D(64, (3, 3), activation='relu'),
#         layers.MaxPooling2D((2, 2)),
#         layers.Conv2D(128, (3, 3), activation='relu'),
#         layers.Flatten(),
#         layers.Dense(256, activation='relu'),
#         layers.Dense(num_classes, activation='softmax'),
#     ])
#     return model

# model = build_model(num_classes=len(label_names))
# model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
# Build CNN model
def build_model(num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(num_classes, activation='softmax'),
    ])
    return model

model = build_model(num_classes=len(label_names))
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:
# Train model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32)


Epoch 1/20


: 

In [ ]:
import cv2 
  
  
# define a video capture object 
vid = cv2.VideoCapture(0) 
  
while(True): 
      
    # Capture the video frame 
    # by frame 
    ret, frame = vid.read() 
  
    # Display the resulting frame 
    cv2.imshow('frame', frame) 
      
    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
  
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 

In [ ]:
import cv2
import numpy as np

def real_time_recognition(model, label_names):
    # Initialize the webcam (use 0 as the parameter to select the default webcam)
    cap = cv2.VideoCapture(0)
    
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
    
    while True:
        ret, frame = cap.read()  # Read a frame from the webcam
        if not ret:
            break
        
        # Preprocess the frame for prediction (resize, expand dimensions, normalize)
        face = cv2.resize(frame, (224, 224))  # Assuming your model expects 224x224 inputs
        face = np.expand_dims(face, axis=0)  # Add batch dimension
        face = face / 255.0  # Normalize pixel values to [0, 1] as during training
        
        prediction = model.predict(face)  # Make prediction
        predicted_class = label_names[np.argmax(prediction)]  # Get the name of the predicted class
        
        # Display the predicted class on the frame
        cv2.putText(frame, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Show the frame
        cv2.imshow('Real-time Face Recognition', frame)
        
        # Break the loop when the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the webcam and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Assuming 'model' is your trained model and 'label_names' is a list of class names
real_time_recognition(model, label_names)
